In [12]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
train_tape = "Z:/rppg/scamps_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [14]:
phys_net = PhysNet(norm='layer_frozen')
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')
phys_net.summary()

Flops per frame:54630323
Model: "phys_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_22 (Sequential)   (None, 128, 32, 32, 16)   1472      
_________________________________________________________________
sequential_23 (Sequential)   (None, 128, 16, 16, 32)   14112     
_________________________________________________________________
sequential_24 (Sequential)   (None, 128, 16, 16, 64)   55616     
_________________________________________________________________
sequential_25 (Sequential)   (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_26 (Sequential)   (None, 64, 8, 8, 64)      110784    
_________________________________________________________________
sequential_27 (Sequential)   (None, 32, 4, 4, 64)      110720    
_________________________________________________________________
sequential_28 (Sequential)   (N

In [16]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet-LN_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet-LN_SCAMPS.h5')

Epoch 1/3
374/374 [==============================] - 272s 710ms/step - loss: 0.6642 - val_loss: 0.4380

Epoch 00001: val_loss improved from inf to 0.43804, saving model to ../weights\PhysNet-LN_SCAMPS.h5
Epoch 2/3
374/374 [==============================] - 288s 751ms/step - loss: 0.3578 - val_loss: 0.2940

Epoch 00002: val_loss improved from 0.43804 to 0.29400, saving model to ../weights\PhysNet-LN_SCAMPS.h5
Epoch 3/3
374/374 [==============================] - 287s 751ms/step - loss: 0.2540 - val_loss: 0.2290

Epoch 00003: val_loss improved from 0.29400 to 0.22904, saving model to ../weights\PhysNet-LN_SCAMPS.h5


In [4]:
phys_net.load_weights('../weights/PhysNet-LN_SCAMPS.h5')

In [17]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_SCAMPS_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_SCAMPS_PURE.h5')

100%|██████████| 59/59 [00:28<00:00,  2.06it/s]


{'Sliding window': {'MAE': 10.091, 'RMSE': 22.04, 'R': 0.62583},
 'Whole video': {'MAE': 10.236, 'RMSE': 21.737, 'R': 0.64064}}

In [18]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet-LN_SCAMPS_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet-LN_SCAMPS_UBFC.h5')

100%|██████████| 42/42 [00:19<00:00,  2.18it/s]


{'Sliding window': {'MAE': 1.715, 'RMSE': 5.024, 'R': 0.95604},
 'Whole video': {'MAE': 1.333, 'RMSE': 2.633, 'R': 0.98832}}